# Create rotations chart data

# Setup

In [1]:
import pandas as pd
import numpy as np
import nba_api
import re

In [2]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
from nba_api.stats.endpoints import LeagueGameFinder
from nba_api.stats.endpoints import PlayByPlayV2
from nba_api.stats.endpoints import PlayByPlay
from nba_api.stats.endpoints import BoxScorePlayerTrackV2

## Import data sheet for home/away

In [3]:
spurs_file = 'data/team_box_scores_spurs.json'
opp_file = 'data/team_box_scores_opp.json'

spurs_df = pd.read_json(spurs_file)
opp_df = pd.read_json(opp_file)

spurs_home = True

In [4]:
spurs_df.head()

,TEAM_ABBREVIATION,home,PTS,FG,FG_PCT,2P,3P,FG3_PCT,FT,FT_PCT,AST,REB,DREB,OREB,BLK,STL,TO,PF
0,SAS,False,131,48-95,50%,38-70,10-25,40%,25-29,86%,28,48,40,8,8,8,10,18


In [5]:
opp_df.head()

,TEAM_ABBREVIATION,home,PTS,FG,FG_PCT,2P,3P,FG3_PCT,FT,FT_PCT,AST,REB,DREB,OREB,BLK,STL,TO,PF
0,MEM,True,119,49-95,52%,39-68,10-27,37%,11-18,61%,27,42,37,5,8,6,11,20


In [6]:
if spurs_df.home.iloc[0] == 'away':
    spurs_home = False
else:
    spurs_home = True

In [7]:
spurs_info = find_teams_by_nickname('Spurs')
spurs_id = spurs_info[0]['id']

# find all Spurs games
gamefinder = LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
team_box_scores_df = gamefinder.get_data_frames()[0]
team_box_scores_df.head(n=10)

spurs_games = list(team_box_scores_df.GAME_ID.unique()) # grab unique list of game IDs

test_game = spurs_games[0]

## Get list of active players and starters for each team

In [8]:
# get list of rosters and starting players
roster_data_df = BoxScorePlayerTrackV2(game_id=test_game).data_sets[0].get_data_frame()
starters_df = roster_data_df[roster_data_df.START_POSITION.isin(['F', 'G', 'C'])]
spurs_starters = list(starters_df[starters_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_starters = list(starters_df[starters_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)
spurs_roster = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_roster = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)

In [9]:
pbp = PlayByPlayV2(game_id=test_game).data_sets

In [10]:
pbp_df = pbp[0].get_data_frame()
pbp_df.head(n=5)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0021901314,2,12,0,1,6:38 PM,12:00,None,None,None,...,None,None,0,0,None,NaN,None,None,None,0
1,0021901314,4,10,0,1,6:38 PM,12:00,Jump Ball Bradley vs. Poeltl: Tip to Ingles,None,None,...,Spurs,SAS,4,204060,Joe Ingles,1.610613e+09,Utah,Jazz,UTA,1
2,0021901314,7,1,47,1,6:38 PM,11:46,Clarkson 7' Turnaround Jump Shot (2 PTS) (Mitc...,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,1
3,0021901314,9,2,78,1,6:38 PM,11:31,O'Neale BLOCK (1 BLK),None,MISS Samanic 4' Floating Jump Shot,...,None,None,4,1626220,Royce O'Neale,1.610613e+09,Utah,Jazz,UTA,1
4,0021901314,11,4,0,1,6:38 PM,11:28,Bradley REBOUND (Off:0 Def:1),None,None,...,None,None,0,0,None,NaN,None,None,None,1


# Functions

In [11]:
def conv_time_to_sec(time):
    """
    This function takes a string quarter time marker and converts it to elapsed game time in seconds.
    Input example: '7:12'
    Output example: 288
    """
    split_time = time.split(':')
    # convert time to seconds
    # seconds in elapsed time
    new_time = int(split_time[0]) * 60 + int(split_time[1])
    new_time = 12*60 - new_time
    return new_time

In [12]:
def get_col(spurs_or_opp):
    """ 
    This function determines whether the spurs or opponent is the home team and returns the 
    appropriate regex player set match and dataframe column.
    
    spurs_or_opp can be 'spurs' or 'opp'
    """
    text_col = '{}DESCRIPTION'
    # check home or away
    if spurs_or_opp == 'spurs':
        re_match = spurs_players_re
        if spurs_home == True:
            side = text_col.format('HOME')
        elif spurs_home == False:
            side = text_col.format('VISITOR')
    elif spurs_or_opp == 'opp':
        re_match = opp_players_re
        if spurs_home == True:
            side = text_col.format('VISITOR')
        else:
            side = text_col.format('HOME')
    
    return side, re_match

In [13]:
def create_period_starters_list(period, df, spurs_or_opp):
    """
    This function returns the 5 players which started a certain period.
    It works by acquiring a unique list of players appearing in play by play then
    returning the first 5 names. 
    
    spurs_or_opp = 'spurs' or 'opp'
    period is an integer figure between 2-4
    df = play by play dataframe
    """
    data = list()
    side, re_match = get_col(spurs_or_opp)
    period_pbp = df[(df.PERIOD == period) & (pbp_df[side].notnull())]
    
    num_rows = period_pbp[side].shape[0]
    
    for i in range(num_rows):
        play_descrip = period_pbp[side].iloc[i]
        try:
            player_match = re_match.search(play_descrip).group()
            if player_match in data:
                continue
            else:
                data.append(player_match)
        except:
            continue
    
    return data[:5]

In [14]:
def split_player_out(pbp):
    """
    Takes in a substitution play by play line and outputs the last name of the player exiting the game.
    """
    split_pbp = pbp.split('FOR')
    return split_pbp[1].strip() # remove all white space

In [15]:
def split_player_in(pbp):
    """
    Takes in a sub play by play line and outputs the last name of the player entering the game.
    """
    split_pbp = pbp.split('FOR')
    return split_pbp[0][4:].strip() # remove white space & 'SUB: '

In [16]:
def round_game_time(time_mark):
    """
    This function takes a time mark in seconds as an input and outputs 
    a rounded minute figure. 
    
    Inputs should be converted to seconds before use. 
    
    Outputs are rounded to the nearest 30 second marker.
    
    Example: 
    Input: 238
    Output: 240
    """
    minute_markers = [i*60 for i in range(1, 49)]
    for i in range(len(minute_markers)):
        diff = abs(minute_markers[i] - time_mark)
        if diff <= 30:
            return minute_markers[i]
        elif diff <= 60:
            return minute_markers[i+1]
        else:
            continue

# Parse PBP data

## Gather quarterly starters

In [17]:
# Create regex of both rosters
spurs_last_names = [' '.join(l_name.split(' ')[1:]) for l_name in spurs_roster]
spurs_re = "(" +  ')|('.join(spurs_last_names) + ")"
opp_last_names = [' '.join(l_name.split(' ')[1:]) for l_name in opp_roster]
opp_re = '(' + ')|('.join(opp_last_names) + ')'

In [18]:
# compile regex patterns
spurs_players_re = re.compile(spurs_re)
opp_players_re = re.compile(opp_re)

In [28]:
spurs_players_re

re.compile(r'(Johnson)|(Samanic)|(Poeltl)|(Walker IV)|(Murray)|(Belinelli)|(Weatherspoon)|(Eubanks)|(Metu)|(DeRozan)|(Gay)|(Mills)|(White)',
re.UNICODE)

In [19]:
# Get all quarter starters
spurs_second_starters = create_period_starters_list(2, pbp_df, 'spurs')
spurs_third_starters = create_period_starters_list(3, pbp_df, 'spurs')
spurs_fourth_starters = create_period_starters_list(4, pbp_df, 'spurs')

opp_second_starters = create_period_starters_list(2, pbp_df, 'opp')
opp_third_starters = create_period_starters_list(3, pbp_df, 'opp')
opp_fourth_starters = create_period_starters_list(4, pbp_df, 'opp')

In [27]:
spurs_second_starters

[]

## Get sub data

In [20]:
# Get appropriate column for each team
spurs_col = get_col('spurs')[0]
opp_col = get_col('opp')[0]

In [21]:
# Grab sub column 
spurs_sub_df = pbp_df[
    (pbp_df[spurs_col].notnull()) & # remove None values
    (pbp_df[spurs_col].str.contains('SUB')) # query if it contains subs
][['GAME_ID', 'PERIOD', 'PCTIMESTRING', spurs_col]].copy()

opp_sub_df = pbp_df[
    (pbp_df[opp_col].notnull()) &
    (pbp_df[opp_col].str.contains('SUB'))
][['GAME_ID', 'PERIOD', 'PCTIMESTRING', opp_col]].copy()

In [29]:
pbp_df[
    (pbp_df[spurs_col].notnull()) & # remove None values
    (pbp_df[spurs_col].str.contains('SUB')) # query if it contains subs
][['GAME_ID', 'PERIOD', 'PCTIMESTRING', spurs_col]].copy()

,GAME_ID,PERIOD,PCTIMESTRING,HOMEDESCRIPTION
45,0021901314,1,6:16,SUB: Niang FOR Clarkson
46,0021901314,1,6:16,SUB: Oni FOR Ingles
67,0021901314,1,3:51,SUB: Ingles FOR Bradley
68,0021901314,1,3:51,SUB: Davis FOR O'Neale
69,0021901314,1,3:51,SUB: Clarkson FOR Mitchell
70,0021901314,1,3:51,SUB: Morgan FOR Oni
140,0021901314,2,9:15,SUB: Mitchell FOR Ingles
141,0021901314,2,9:15,SUB: O'Neale FOR Morgan
142,0021901314,2,9:15,SUB: Oni FOR Niang
155,0021901314,2,8:24,SUB: Brantley FOR Clarkson


In [22]:
# Assign total game time
spurs_sub_df['game_time'] = spurs_sub_df.PCTIMESTRING.apply(conv_time_to_sec) + (spurs_sub_df.PERIOD - 1) * (12*60)
opp_sub_df['game_time'] = opp_sub_df.PCTIMESTRING.apply(conv_time_to_sec) + (opp_sub_df.PERIOD - 1) * (12*60)

In [23]:
# Assign columns for player in and out
spurs_sub_df['player_in'] = spurs_sub_df[spurs_col].apply(split_player_in)
spurs_sub_df['player_out'] = spurs_sub_df[spurs_col].apply(split_player_out)

opp_sub_df['player_in'] = opp_sub_df[opp_col].apply(split_player_in)
opp_sub_df['player_out'] = opp_sub_df[opp_col].apply(split_player_out)

In [24]:
spurs_sub_df.columns

Index(['GAME_ID', 'PERIOD', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'game_time',
       'player_in', 'player_out'],
      dtype='object')

In [25]:
# Filter out unnecessary columns
# Remove time string and pbp column
spurs_sub_df = spurs_sub_df[['GAME_ID', 'PERIOD', 'game_time', 'player_in', 'player_out']]
opp_sub_df = opp_sub_df[['GAME_ID', 'PERIOD', 'game_time', 'player_in', 'player_out']]

In [26]:
spurs_sub_df

,GAME_ID,PERIOD,game_time,player_in,player_out
45,0021901314,1,344,Niang,Clarkson
46,0021901314,1,344,Oni,Ingles
67,0021901314,1,489,Ingles,Bradley
68,0021901314,1,489,Davis,O'Neale
69,0021901314,1,489,Clarkson,Mitchell
70,0021901314,1,489,Morgan,Oni
140,0021901314,2,885,Mitchell,Ingles
141,0021901314,2,885,O'Neale,Morgan
142,0021901314,2,885,Oni,Niang
155,0021901314,2,936,Brantley,Clarkson


# Create rotations data for visualization

In [71]:
# create a dictionary with players as keys and sub in times
# create a dict of sub out times
# clean up times to fit into one minute bins
# minute start times also become dataframe columns
# check starters for quarter start times - 0, 720, 1440, 2160

spurs_subs_in = dict()
spurs_subs_out = dict()
opp_subs_in = dict()
opp_subs_out = dict() 

# A list of quarter start times in seconds
quarter_start_times = [i*720 for i in range(0, 4)]

In [89]:
# append the rounded game time to a new column
spurs_sub_df['rounded_game_time'] = spurs_sub_df.game_time.apply(round_game_time)
opp_sub_df['rounded_game_time'] = opp_sub_df.game_time.apply(round_game_game)

In [91]:
# 1. add quarter starts 
# 2. go through each time to add 1 or 0

spurs_quarter_starts = [spurs_starters, spurs_second_starters, \
                       spurs_third_starters, spurs_fourth_starters]

opp_quarter_starts = [opp_starters, opp_second_starters, \
                     opp_third_starters, opp_fourth_starters]

for i in range(4):
    

# decide if sub out equates to 1 or 0
# same with sub in

In [109]:
def lookup_full_name(last_name, roster):
    """
    This function takes a player's last name and outputs his full name.
    
    The input should also include a roster list of full names.
    
    Example:
    Input: Johnson
    Output: Keldon Johnson
    """
    for name in roster:
        if name.endswith(last_name):
            return name
        else:
            continue

In [119]:
spurs_sub_df.player_in.apply(lookup_full_name, args=(spurs_roster, ))

36            Marco Belinelli
52     Quinndary Weatherspoon
71               Drew Eubanks
80           Lonnie Walker IV
81              Chimezie Metu
98            Dejounte Murray
143            Keldon Johnson
144          Lonnie Walker IV
177              Jakob Poeltl
178           Marco Belinelli
192           Dejounte Murray
200              Drew Eubanks
210              Luka Samanic
288           Marco Belinelli
290             Chimezie Metu
309              Drew Eubanks
338           Dejounte Murray
342              Luka Samanic
391            Keldon Johnson
392          Lonnie Walker IV
424              Drew Eubanks
425              Luka Samanic
442           Dejounte Murray
Name: player_in, dtype: object

In [121]:
spurs_sub_df.player_out.apply(lookup_full_name, args=(spurs_roster, ))

36               Luka Samanic
52           Lonnie Walker IV
71               Jakob Poeltl
80             Keldon Johnson
81            Dejounte Murray
98     Quinndary Weatherspoon
143           Dejounte Murray
144           Marco Belinelli
177            Keldon Johnson
178              Drew Eubanks
192              Luka Samanic
200             Chimezie Metu
210          Lonnie Walker IV
288           Dejounte Murray
290              Luka Samanic
309              Jakob Poeltl
338          Lonnie Walker IV
342            Keldon Johnson
391              Drew Eubanks
392              Luka Samanic
424           Dejounte Murray
425           Marco Belinelli
442             Chimezie Metu
Name: player_out, dtype: object

In [103]:
# Apply full names to player in and player out

'Walker IV'